# Threads
* Reading: OS essentials Page 163-190
* Abstraction of a processor
* a *thread of control*
### Why threads
* A natural means for dealing with concurrency:
  * multiprocessor and equally useful on uniprocesors
### Processes vs. Threads
* Both provide concurrency
* What the difference?
  * Two single-thread processes vs. one two-threaded process
    * First, for an existing process, it is much cheaper to add a thread than creating a new process
    * Context switching: it is also cheaper sharing context between two threads in the same process

# Matrix Multiplication
* C=A*B
* A: m x n
  * 2 x 5
* B: n x p
  * 5 x 3
* C: ?
  * 2 x 3
  * m x p


What the heck

```c
#include <stdio.h>
#include <pthread.h>
#include<string.h>

#define M 3
#define N 4
#define P 5

long A [M][N]
long B [N][P]
long C [M][P]


long A[m]n], B[n][p], C[m][p];
for (i=0; i < m; i ++) // create worker threads
{
    pthread_create(&thr[i],0, matmult, i);
    // &thr[i] is the id of the thread 
    // 0 is the 
    // matmult is the task, 
    // i is the parameter you need to provice for the function
}

```

# Example Question
```c
#include "csapp.h"
#define N 2
void *thread(void *vargp);

char **ptr; /* Global variable */ // line:conc:sharing:ptrdec

int main()
{
    int i; // stored in the stack for the main thread
    pthread_t tid; // stored in the stack for the main thread
    char *msgs[N] = {
        "Hello from foo",
        "Hello from bar"};
    ptr = msgs;                 // since ptr is a global variable, every peer thread can access whatever it points to
                                // so now everything can access msgs
    for (i = 0; i < N; i++)
    {
        Pthread_create(&tid, NULL, thread, (void *)i);
    }
    Pthread_exit(NULL); // waits for all other threads to finsih
}

void *thread(void *vargp)
{
    int myid = (int)vargp;
    static inc cnt = 0;                                    // line:conc:sharing:cntdec
    // so this is static, so it is shared
    printf("[%d]: %s (cnt=%d)\n", myid, ptr[myid], ++cnt); // line:conc:sharing:stacj
    return NULL;
}
```

## What is accessed
| Variable Instance | Main Thread | Peer Thread 0 | Peer Thread 1 |
| ----------------- | ----------- | ------------- | ------------- |
| ptr               | yes         | yes           | yes           |
| cnt               | no          | yes           | yes           |
| msg.m             | yes         | yes           | yes           |
| i.m               | yes         | no            | no            |
| myid.p0           | no          | yes           | no            |
| myid.p1           | no          | no            | yes           |

## Thread
Has its own:
* Thread ID
* Stack
* Stack Pointer ($rsp)
* Program Counter ($rip)
* FLAG (Condition Codes)
* General Purpose Registers

Sharing:
* Virtual Address Space
* Code
* Data
* Heap
* Open Files

```c
pthread_create([], 0 task, arg); 
pthread_join(); // there is some information mantained, like how it is terminated
// similar to wait()
pthread_exit(); // waits for all other peers to finish

wait(); // this must be called if you want the child to do something, and THEN the parent does something.
// it "waits" until the child is done
// wait vs waitpid, you use waitpid if there are multiple children (you specify which pip/child to wait for)
// also use waitpid if the parent doesnt terminate. this is becasue there wont be any chance for the init process to cleanup since it never terminates
// ^ like a shell or server
```

### Contex
* When you switch between threads, there are a few things of info you need to keep and tell the next thread.
  * Thread ID
  * Stack
  * Stack Pointer
  * Program Counter
  * Condition codes
  * General-purpose Register Values

### Data Sharing
* Each thread shares the rest of the process context with other threads:
  * The entire user virtual address space
  * Read-only text (code)
  * Read/Write data
  * Heap
  * Same set of open files
  * Library Code

### Strack SIze (2MB by default)
```c
pthread_t thread;
pthread_attr_t thr_attr;
pthread_attr_init(&thr_attr);
pthread_attr_setstacjsize(&thr_attr,20*1024*1024); // this is 20MB
...
pthread_create(&thread, &thr_attr, startroutine, arg);
...
```

### Mutexes
* A synchronization construct providing mutual exclusion
* Code locking:
  * only one thread is executoing a particular piece of code at once
* Data locking
  * only one htread is accessing a particular thread at once
* A mutexe must be initialized

```c
pthread_mutex_t m = PTHREAD_MUTEX_INITIALIZER;
int x;
Pthread_mutex_lock(&m);
x = x+1;
```
* An important restriction:
  * the thread that locked a mutex should be the thread that unlocks it.

```c
pthread_mutex_lock(&m);
// critical selection
pthread_mutex_unlock(&m)
```

### Singly Linked List
```c
typedef struct node{
    pthread_mutex_t mutex;
    int value;
    struct node *next;
} node_t;
pthread_mutex_t global_mutex;

void add_after (node_t *after, node_t *new){
    pthread_mutex_lock(&after->mutex);
    new->next = after->next;
    after->next = new;
    pthread_mutex_unlock(&global_mutex);
    


```

### Doubly Linked List
* there is one more pointer inside each node  

if you:
* add one in middle: there will be 4 operations
* remove one in the middle: 2 operations
```c
// can cause deadlock
void add_after(node_t *after, node_t *new){
    pthread_mutex_lock(&after->mutex);
    pthread_mutex_lock(&after->next->mutex);
    after->next->prev = new;
    new->next = after->next;
    new->prev = after;
    after->next = new;
    pthread_mutex_unlock(&after->next->mutex);
    pthread_mutex_unlock(&after->mutex);
}
void delete(npde_t *old){
    pthread_mutex_lock(&old->prev->mutex);
    pthread_mutex_lock(&old->mutex);
    pthread_mutex_lock(&old->next->mutex);
    old->prev->next = old->next;
    old->next->prev = old->prev;
    pthread_mutex_unlock(&old->next->mutex);
    pthread_mutex_unlock(&old->prev->mutex);
    pthread_mutex_unlock(&old->mutex);
}
```

```c
proc1 (){
    pthread_mutex_lock(&m1);
    // use object 1
    pthread_mutex_lock(&m2);
    // use objects 1 and 2
    pthread_mutex_unlock(&m1);
    pthread_mutex_unlock(&m2);
}

proc2 (){
    while(1){
        pthread_mutex_lock(&m2);
        if(!pthread_mutex_trylock(&m1)){

            if(pthread_mutex_trylock(&m3)){
                break;
            }
        }
        pthread_mutex_unlock(&m2);
    }
    // use objects 1 and 2
    pthread_mutex_unlock(&m1);
    pthread_mutex_unlock(&m2);
}




# Exam 2 on November 9th or 11th

# Multi-thread Concurrency
There are several different OS problems that we often talk about
1. Dining Philosophers:
    * 5 Philosophers, 5 Chopsticks
    * In the middle is a plate of spaghetti
    * They can either
      * think (like a philosopher does)
      * eat
      * sleep
    * If they eat, they need to pick up two chopsticks
    * you dont want them to starve
    * so its all about managing the timing for eating and sleeping (and thinking)
2. Producer - Consumer
   * a producer makes stuff, while a consumer consumes it.
   * Fir the producer to add onto the array
     * EMPTY
     * since we dont want them to overwrite any data
     * So there is a guard
   * For the consumer to take away something on the array
     * OCCUPIED
     * since we dont want them to overflow the array
3. Reader - Writer Problemn
   * Semaphore : `integer >= 0`
     * binary
     * accumulate



`P(S): S=S-1;`  
`V(S): S=S+1;` 

### Producer - Consumer (cont.)
```c
Semaphore empty = BSIZE;
Semaphore occupied = 0;
int nextin = 0;
int nextout = 0;

char consume(){
    char item;
    P(occupied); // the condition to start
    item = buf[nextout];
    if(++nextout >= BSIZE)
        nextout = 0;
    V(empty);
    return item;
}

void produce(char item){
    P(empty); // the condition to start
    buf[nextitem] = item;
    if(++nextin >= BSIZE)
        nextin = 0;
    V(occupied);
}
```

```c
#include<semaphore.h>

int sem_

```

# Project 5
```
main (a1,a2,a3)
* create producer threads: a2
* create consumer threads: a3
* sleep: a1 (how long)
* terminate

```
```c
void *producer(void *param)
{
    while(1){
        sleep() // mess around and try out different times
        // generate a random number, set the value to item
        insert_item(item); 
        // ^ like the voice produce item earlier in this
    }
}

insert_item:
    P(s);
    lock(m);
    
    insert the stuff from earlier with the buf[nextin] etc...
    
    unlock(m);
    V(s);

remove_item:
    P(s)
    lock(m)
        like the insert_item but uses consume instead of produce
    unlock(m)
    V(s)
```


`./a.out 15, 3, 3`

# Reader & Writer
* Accessing the same datastructure, like a database
* problem: only 1 writer is allowed to be active (modifying the data structure)
  * no other writers, no readers)
* however, a non-problem: multiple readers are allowed to be active at a time
  * but at that time, no w3riters are allowed

```c
#define writer:
while(true){
    sem_wait(s);
    
    // "critical section for writing"
    
    sem_post(s);
}

#define Readers: reader_num;
while(true){
    lock(m);
    reader_num++;

    if(reader_num == 1) {
        sem_wait(s); // if it is the frist one, make sure to lock it
    }
    unlock(m);
    // "critical section for reading"
    lock(m);
    reader_num--;
    if(reader_num == 0){
        sem_post(s); // if it is the last one to stop, make sure to unlock it
    }
    unlock(m);
}


### Exam Topics:
1. Some CPU Scheduling
2. Mutex functions
3. Semaphores and Producer/consumer 
4. Dining Philosophers
5. Fill in the blank code, semaphores and thread creation

## Cache: 
* num blocks * block size = size of cache
* Hit ratio = hit/successes
* Miss ratio = 1-hit ratio
* Avg access time: Hit time + Miss ratio * Miss penalty
* Miss penalty = time wasted after miss in cache

### Direct Mapped Cache
* Block address in memory % blocks in cache (to determine location)
  * mem addr.   cache index.
  * 10101 % 8 = 101 (5)
  * 11001 % 8 = 001 (1)
* Only one memory location for each block in cache
* We need to store the high order bits in order to distinguish the blocks in memory ( Tags ) mapping 101 -> 10101
  * Valid bit: 1 = present, 0 = not present (initial value)
  * If is a miss, copy memory block into the index
* Inserting:
WordAddress Valid Tag Data



* Memory address to block address:  
  * $\lfloor\frac{Mem. Addr.}{Block Size}\rfloor$
  * mem address divided by block size rounded down

the last 4 bits range from 0 to 1111, with the higher bits specifiying what block it is  
16B -> 4 subblocks because its 16=2^4

# Cache design for direct map (KNOW THIS)
1. Assume the current cache has 64 blocks, and 16 blocks/byte. When a program in execution needs the data from Memory address 1200.
   1. Which cache index will the data be put into?
      1. 1200 / 16 (floor) = 75. This is the block address
      2. **75 % 64 = 11. This is the cache index**
   2. draw a diagram of 32-bit address space, show the Tag, Index, and Offset field.


0->3: Byte offset   
4->9: Index

| Tag(bits)   | index    | byte-offset |
| ----------- | -------- | ----------- |
| 22 (10->31) | 6 (4->9) | 3 (0->3)    |

<br>

| bit    | cache |
| ------ | ----- |
| 000000 | value |
| 000001 | value |
| ...    | ...   |
| 111111 | value |

block addr. mod 64(2^6). Meaning the remainder is the index field (which will be 6 bits)

the number of cache indexes, of the number of cache blocks will determine the field of *index*

Index field, is determined by the number of blocks. if the number of blocks is 1024 (2^10), then the index field is 10.

the tag field goes until 31 (usually. all that we covered did)

$\lfloor\frac{Memory Address}{Block Size}\rfloor = $ Block Address  
Block Address % # of blocks in cache = Cache Index  
|Tag |Index| Offset|
|-|-|-|

* Block address = Tag-Index
* Index has `m` bits
  * the # of blocks in cache = `2^m`
* offset has `t` bits
  * block size = `2^t`
    * the block size refers to the size of a single block in memory **NOT THE SIZE OF THE BLOCK ADDRESS**

<br>

* "The offset will tell you in which spot of the block will have 1200"
* We are trying to access 1200, it's a miss because it is not in the cache
* so we copy block 75 to the cache
* now when we are accessing 1204 it is a hit, because it is available in the cache



# hw 4
* 5 bits in the offset and 5 bits in the index 

| tab |index|offset |
|-----|-----|-------|
|31-10| 9-5 |  4-0  |

* what is the cache block size: 2^5 (2^(offset))
* how many entries (indexes_ does the cache have? 2^5 (2^(index))

|0|4|16|132|232|160|1024|32 |
|-|-|--|---|---|---|----|---|

* for each memory address, which index in the cache will CPU look for? Indicate hit/miss for each access
  * 0
    * memory address to block address
      * floor of memory address divided by block size
        * we find the size of to be 32 since it is 2^t (2^5)
      * BA: floor(0/32) = `0`
    * then block address to cache index
      * block address % # of blocks in cache
        * we find the # of blocks to be 32 since it is 2^m (2^5)
      * CI: `0` mod 32 = 0
  * 4
    * BA: 0
    * CI: 0
  * 16
    * BA: 0
    * CI: 0
  * 132
    * BA: `4` (floor(132 / 32))
    * CI: 4 (`4` mod 32 = 4)
  * 1024
    * BA: `32` (floor(1024 / 32))
    * CI: 0 (`32` mod 32 = 0)

0: miss (adds block to cache)  
4: hit (included in block 1)  
16: hit (included in block 1)  
132: miss (replaces block 1 with block n in cache)  
1024: miss (replaces block n with block m in cache)  


hit ratio = # of huts / total num of accesses


if we were to access 4 after all of these, it would be a miss because by that time, that block will be out of cache

# Exam topics:
* CPU Scheduling
  * MLQ
  * MLFQ
* Thread vs Process
  * Thread shares: Data, Code (Text), Heap
    * Has its own stack
    * Create, Terminate threads
      * `pthread_exit`, `pthread_create`
      * `pthread_join` - waits for the other threads
    * Exit or return terminates the WHOLE process, all the threads
  * Process contains complete image of memory
* Mutex
  * `lock()`, `unlock()`
  * The problem of Deadlock
    * how to solve it
      * define an order
      * `trylock()`
* Semaphore
  * Integer, only positive num or 0
  * `P()`
  * `V()`
  * Ideas:
    * Dining philosopher
    * Reader/Writer
  * producer/consumer
* Cache/Direct map